In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from folium import GeoJson, GeoJsonTooltip
import folium
import geopandas as gpd
from folium.plugins import GroupedLayerControl, MarkerCluster
from folium.plugins import HeatMap
from branca.colormap import linear
import osmnx as ox
from shapely.geometry import LineString, MultiLineString
import overpy
from shapely.geometry import LineString
from rasterstats import zonal_stats
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Load the shapefile using GeoPandas
data_folder = "../Data/UAE_Shapefile/gadm41_ARE_shp"
shapefile_name = "gadm41_ARE_3.shp"
shapefile_path = os.path.join(data_folder, shapefile_name)
print("Shapefile path => ", shapefile_path)
gdf = gpd.read_file(shapefile_path)
print("Columns : ", gdf.columns.tolist())

Shapefile path =>  ../Data/UAE_Shapefile/gadm41_ARE_shp\gadm41_ARE_3.shp
Columns :  ['GID_3', 'GID_0', 'COUNTRY', 'GID_1', 'NAME_1', 'NL_NAME_1', 'GID_2', 'NAME_2', 'NL_NAME_2', 'NAME_3', 'VARNAME_3', 'NL_NAME_3', 'TYPE_3', 'ENGTYPE_3', 'CC_3', 'HASC_3', 'geometry']


In [4]:
# Abu Dhabi City
abu_dhabi_gdf = gdf[gdf["NAME_1"] == "Abu Dhabi"]
print(f"Shape: {abu_dhabi_gdf.shape}")
abu_dhabi_gdf.head(5)

Shape: (125, 17)


,GID_3,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,ARE.1.1.2_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Abu Al Abyad,NA,NA,Municipality,Municipality,NA,NA,"MULTIPOLYGON (((53.87441 24.26483, 53.87717 24..."
1,ARE.1.1.3_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Abu Dhabi Island,NA,NA,Municipality,Municipality,NA,NA,"MULTIPOLYGON (((54.28496 24.4461, 54.28496 24...."
2,ARE.1.1.4_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Airport District,NA,NA,Municipality,Municipality,NA,NA,"POLYGON ((54.61171 24.45358, 54.61182 24.45385..."
3,ARE.1.1.5_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Al Falah,NA,NA,Municipality,Municipality,NA,NA,"POLYGON ((54.67202 24.40866, 54.67627 24.41914..."
4,ARE.1.1.6_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Al Jubail Island,NA,NA,Municipality,Municipality,NA,NA,"POLYGON ((54.52723 24.51838, 54.52185 24.51998..."


Calculate the Midpoint Coordinates

In [5]:
# total_bounds → (minx, miny, maxx, maxy) for the entire GeoDataFrame
minx, miny, maxx, maxy = abu_dhabi_gdf.total_bounds

# x corresponds to longitude, y corresponds to latitude
sw_lat = miny  # southernmost lat
sw_lon = minx  # westernmost lon
ne_lat = maxy  # northernmost lat
ne_lon = maxx  # easternmost lon

print("Overall Bounding Box for KSA Polygons:")
print(f"  SW corner (lat, lon): ({sw_lat}, {sw_lon})")
print(f"  NE corner (lat, lon): ({ne_lat}, {ne_lon})")

# Calculate midpoint coordinates
mid_lat = (sw_lat + ne_lat) / 2
mid_lon = (sw_lon + ne_lon) / 2

Overall Bounding Box for KSA Polygons:
  SW corner (lat, lon): (22.631621401000018, 51.49797800000016)
  NE corner (lat, lon): (25.252622605000056, 56.018131256000174)


Get Roadways from OpenStreetMap

In [ ]:
# Initialize theoverpy API
api = overpy.Overpass()

all_clipped_roads = []  # Will hold a GeoDataFrame for each geometry

for idx, row in abu_dhabi_gdf.iterrows():
    geom = row.geometry  # Polygon or MultiPolygon

    # Get bounding box from that geometry
    minx, miny, maxx, maxy = geom.bounds
    bbox_str = f"{miny},{minx},{maxy},{maxx}"

    # Overpass query: bounding box style
    query = f"""
    (
      way["highway"~"motorway|trunk|primary|secondary|tertiary"]({bbox_str});
    );
    (._;>;);
    out body;
    """

    try:
        # Query Overpass for this bounding box
        result = api.query(query)
    except Exception as e:
        # If any query fails, we log it and continue, so the loop doesn't stop
        print(f"Overpass query failed for geometry index {idx}: {e}")
        continue  # Skip to next geometry

    # Convert Overpass ways to line features
    roads_data = []
    for way in result.ways:
        coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
        if len(coords) >= 2:
            roads_data.append({
                "geometry": LineString(coords),
                "highway": way.tags.get("highway", "unknown"),
                "name": way.tags.get("name", "Unnamed Road"),
                # Optionally keep track of which polygon this road belongs to
                "polygon_idx": idx
            })

    # If there are no roads, skip
    if not roads_data:
        continue

    gdf_roads = gpd.GeoDataFrame(roads_data, crs=abu_dhabi_gdf.crs)

    # Now clip to the exact polygon or multipolygon so we only keep roads inside it
    gdf_roads_clipped = gpd.clip(gdf_roads, geom)

    # Store in a list to merge later
    all_clipped_roads.append(gdf_roads_clipped)

# Merge all results into one big GeoDataFrame
if len(all_clipped_roads) > 0:
    gdf_highways_clipped = pd.concat(all_clipped_roads, ignore_index=True)
    gdf_highways_clipped = gpd.GeoDataFrame(gdf_highways_clipped, crs=abu_dhabi_gdf.crs)
else:
    # If we got nothing, create an empty GeoDataFrame
    gdf_highways_clipped = gpd.GeoDataFrame(columns=["geometry", "highway", "name", "polygon_idx"], crs=abu_dhabi_gdf.crs)

# Remove duplicates if there is overlap in bounding boxes:
gdf_highways_clipped = gdf_highways_clipped.drop_duplicates(subset="geometry")

In [15]:
# Display gdf_highways_clipped
print("Columns :", gdf_highways_clipped.columns.tolist())
gdf_highways_clipped.head(3)

Columns : ['geometry', 'highway', 'name', 'polygon_idx']


,geometry,highway,name,polygon_idx
0,"LINESTRING (53.90862 24.12192, 53.9088 24.1219...",secondary,Unnamed Road,0
1,"LINESTRING (53.90938 24.12224, 53.90918 24.123...",secondary,Unnamed Road,0
2,"LINESTRING (53.90826 24.12854, 53.90826 24.128...",secondary_link,Unnamed Road,0


In [16]:
# Print unique highway types
gdf_highways_clipped["highway"].value_counts(sort= True)

highway
tertiary          6439
secondary         3311
primary           2083
tertiary_link     1470
primary_link      1386
secondary_link    1370
motorway_link     1353
trunk              758
trunk_link         733
motorway           559
Name: count, dtype: int64

In [17]:
# Dsiplay unique names per highway type
unique_names_per_highway = gdf_highways_clipped.groupby('highway')['name'].unique()
unique_names_per_highway

highway
motorway          [شارع الخَلِيج العَرَبي, شارع الشيخ زايد بن سل...
motorway_link     [Unnamed Road, طريق سويحان, شارع الخَلِيج العَ...
primary           [شارع هزاع بن زايد الأول, شارع الشيخ راشد بن س...
primary_link      [Unnamed Road, شارع أم سلمة, شارع القِيْدُوم, ...
secondary         [Unnamed Road, جسر الحديريات, شارع شخبوط بن سل...
secondary_link    [Unnamed Road, شارع اليَزْوَة, شارع مِمْبَاسِي...
tertiary          [Unnamed Road, شارع خليفة المبارك, شارع مبارك ...
tertiary_link     [Unnamed Road, شارع القتادي, شارع السياب, شارع...
trunk             [شارع الخَلِيج العَرَبي, شارع الشيخ زايد بن سل...
trunk_link        [Unnamed Road, شارع الهَمَايِل, شارع السُّهُول...
Name: name, dtype: object

In [18]:
# Create the map centered on the midpoint with appropriate zoom level
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=9, tiles='cartodbpositron')

# Add the GeoJSON polygon to the map
folium.GeoJson(
    data=abu_dhabi_gdf,
    name='Abu Dhabi Boundary',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_3'],
        aliases=['District: '],
        localize=True
    )
).add_to(m)

for _, row in gdf_highways_clipped.iterrows():
    geom = row.geometry

    # Handle LineString
    if isinstance(geom, LineString):
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(
            locations=coords,
            color='red',
            weight=2,
            tooltip=row["name"]
        ).add_to(m)

    # Handle MultiLineString
    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(lat, lon) for lon, lat in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color='red',
                weight=2,
                tooltip=row["name"]
            ).add_to(m)


# Save the updated map with roadways
m.save('../Data/Maps/Abu_Dhabi_Map_with_Roadways_Search_By_Districts_V1.html')